<a href="https://colab.research.google.com/github/Kshibin/NLP-Project/blob/main/Fake_Job_Offer_And_Scam_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/DATA/Fake_Jobs_Postings.csv"
df = pd.read_csv(path)
print(df)

       job_id                                              title  \
0           1                                   Marketing Intern   
1           2          Customer Service - Cloud Video Production   
2           3            Commissioning Machinery Assistant (CMA)   
3           4                  Account Executive - Washington DC   
4           5                                Bill Review Manager   
...       ...                                                ...   
17875   17876                   Account Director - Distribution    
17876   17877                                 Payroll Accountant   
17877   17878  Project Cost Control Staff Engineer - Cost Con...   
17878   17879                                   Graphic Designer   
17879   17880                         Web Application Developers   

                   location   department salary_range  \
0          US, NY, New York    Marketing          NaN   
1            NZ, , Auckland      Success          NaN   
2           

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Fake_Jobs_Postings.csv to Fake_Jobs_Postings (1).csv


In [ ]:
import pandas as pd

# Load dataset from CSV
df = pd.read_csv('Fake_Jobs_Postings.csv')  # upload from files or provide path
df.head()  # show first 5 rows

# Save cleaned version (later)
df.to_csv('cleaned_fake_jobs.csv', index=False)


In [ ]:
# Drop null rows
df = df.dropna()

# Combine relevant fields
df['text'] = df['title'] + ' ' + df['description']

# Keep only 'text' and 'label' (1 = fake, 0 = real)
df = df[['text', 'fraudulent']]
df.columns = ['text', 'label']


/tmp/ipython-input-32-1004756086.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['title'] + ' ' + df['description']


In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # remove punctuation
    words = text.split()
    words = [word for word in words if word not in stopwords.words('english')]
    return ' '.join(words)

df['clean_text'] = df['text'].apply(clean_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['clean_text'])

y = df['label']


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.92      1.00      0.96       142
           1       0.00      0.00      0.00        13

    accuracy                           0.92       155
   macro avg       0.46      0.50      0.48       155
weighted avg       0.84      0.92      0.88       155



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import joblib

# Save model and vectorizer
joblib.dump(model, 'logistic_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')


['vectorizer.pkl']

In [ ]:
model = joblib.load('logistic_model.pkl')
vectorizer = joblib.load('vectorizer.pkl')


In [ ]:
import joblib

# Load saved model and vectorizer
model = joblib.load('logistic_model.pkl')
vectorizer = joblib.load('vectorizer.pkl')


In [ ]:
def predict_job_message(message):
    # Clean the input text (same as training)
    cleaned = clean_text(message)

    # Vectorize the input
    vec = vectorizer.transform([cleaned])

    # Make prediction
    prediction = model.predict(vec)[0]

    # Return result
    return "Scam" if prediction == 1 else "legit"


In [ ]:
# Example scam job message
msg1 = "IC&amp;E Technician | Bakersfield, CA Mt. PosoPrincipal Duties and Responsibilities:Â Calibrates, tests, maintains, troubleshoots, and installs all power plant instrumentation, control systems and electrical equipment.Performs maintenance on motor control centers, motor operated valves, generators, excitation equipment and motors.Performs preventive, predictive and corrective maintenance on equipment, coordinating work with various team members.Designs and installs new equipment and/or system modifications.Troubleshoots and performs maintenance on DC backup power equipment, process controls, programmable logic controls (PLC), and emission monitoring equipment.Uses maintenance reporting system to record time and material use, problem identified and corrected, and further action required; provides complete history of maintenance on equipment.Schedule, coordinate, work with and monitor contractors on specific tasks, as required.Follows safe working practices at all times.Identifies safety hazards and recommends solutions.Follows environmental compliance work practices.Identifies environmental non-compliance problems and assist in implementing solutions.Assists other team members and works with all departments to support generating station in achieving their performance goals.Trains other team members in the areas of instrumentation, control, and electrical systems.Performs housekeeping assignments, as directed.Conduct equipment and system tagging according to company and plant rules and regulations.Perform equipment safety inspections, as required, and record results as appropriate.Â Participate in small construction projects.Â  Read and interpret drawings, sketches, prints, and specifications, as required.Orders parts as needed to affect maintenance and repair.Performs Operations tasks on an as-needed basis and other tasks as assigned.Available within a reasonable response time for emergency call-ins and overtime, plus provide acceptable off-hour contact by phone and company pager.Â Â Â Â Â Â Â Â Â  Excellent Verbal and Written Communications Skills:Ability to coordinate work activities with other team members on technical subjects across job families.Ability to work weekends, holidays, and rotating shifts, as required."
# Example legit job message
msg2 = "We Are Looking for college interns with the passion to be in the entertainment industry. Industry Mogul Alander Pulliam has put together a youth and motivated program for college students throughout the Los Angeles market. To help college students enrich there knowledge and there experience."
msg3 = "Health Catalyst is recruiting for Data Entry/Administrative Assistant positions with exciting corporate and non-profit clients. some professional internship experience and a willingness to learn from the ground up. These positions require strong MS Office skills, a professional demeanor and an ability to interact with all levels of management and staff. We have opportunities in a variety of environments including legal, corporate, association and both domestic and international non-profit offices. QualificationsA minimum three years of experience in data entry primarily entering numerical research data in a variety of specialized formats. Knowledge of word processing, basic spreadsheet and database maintenance and a working knowledge of MS PowerPoint. Basics of database entry of project management setups would be preferred.Job Requirements*Must be able to work 40 hours/week during normal business hours Mon. to Fri.*1 years in an order entry/data entry/admin assistant /receptionist department*Proficiency in Microsoft Office*MUST TYPE 45 WPM*Excellent written and oral communication skills*Superb tenure in previous positionsThese positions are temp-to-hire and are available immediately. For consideration, please send your resume and cover letter, along with salary requirements to:#EMAIL_4c7d0e2d8114d28ab3dc969e6512ff38f396bd278816d53b41b8e9575f217588#"
#predict
print("Message 1:", predict_job_message(msg1))
print("Message 2:", predict_job_message(msg2))
print("Message 3:", predict_job_message(msg3))


Message 1: Scam
Message 2: legit
Message 3: legit
